In [1]:
import os
import sys

import pandas as pd
import numpy as np

sys.path.insert(0, "\\".join(os.path.abspath(os.curdir).split("\\")[:-2:]))

from pydicom import dcmread

from src.utils.utils import *

In [ ]:
import matplotlib.pyplot as plt

path = "D:\\mathe\\Documents\\BancoDados_IC\\CMMD\\D1-0001\\07-18-2010-NA-NA-79377\\1.000000-NA-70244"
directory = Path(path)
paths_dicom_file = list(directory.rglob("*.dcm*"))
dicom_file = dcmread(paths_dicom_file[0])

img = dicom_file.pixel_array

fig = plt.figure()
plt.imshow(img, cmap='gray')


# Extrair caracteristicas matriz GLCM

CMMD

In [2]:
path_json = "C:\\Users\\mathe\\Dropbox\\PythonNotebooks\\Radiomica\\outputs\\mamografia\\extracted_metadata"
name_json = "extracted_metadata_CMMD"
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
distances = [1, 5]

glcm_df_CMMD = get_glcm_features(name_json, path_json, angles, properties, distances)
glcm_df_CMMD.to_csv("../../outputs/mamografia/matriz_features_glcm_CMMD.csv", index=False)
glcm_df_CMMD.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_5,dissimilarity_ang_45_dist_5,dissimilarity_ang_90_dist_5,dissimilarity_ang_135_dist_5,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_5,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_5,energy_ang_45_dist_5,energy_ang_90_dist_5,energy_ang_135_dist_5,pathology
0,0.625476,0.707031,0.627003,0.705654,0.971535,1.002511,0.962851,1.005881,0.991534,0.989223,...,0.840798,0.917298,0.917229,0.917193,0.917241,0.917121,0.916889,0.916598,0.916950,Benign
1,1.184211,1.307271,1.150881,1.317487,1.731502,1.718066,1.622076,1.770468,0.989185,0.986792,...,0.660684,0.813474,0.813478,0.813445,0.813389,0.813061,0.813188,0.812932,0.812825,Benign
2,1.140270,1.302220,1.141987,1.289631,1.799599,1.927280,1.832726,1.854555,0.991724,0.989122,...,0.770689,0.878428,0.878354,0.878307,0.878340,0.878173,0.877946,0.877569,0.877889,Benign
3,1.682375,1.903251,1.648690,1.878657,2.547731,2.645756,2.406968,2.534075,0.987955,0.984806,...,0.596503,0.772685,0.772592,0.772693,0.772697,0.772142,0.771913,0.772185,0.772336,Benign
4,1.659040,1.898933,1.665683,1.884636,2.448483,2.618151,2.551372,2.555354,0.986040,0.981888,...,0.608706,0.781069,0.780941,0.780920,0.780936,0.780605,0.780213,0.779858,0.780196,Benign


CBIS-DDSM 

In [ ]:
path_json = "C:\\Users\\mathe\\Dropbox\\PythonNotebooks\\Radiomica\\outputs\\mamografia\\extracted_metadata"
name_json = "extracted_metadata_CBIS-DDSM"
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
distances = [1, 5]

glcm_df_CBIS_DDSM = get_glcm_features(name_json, path_json, angles, properties, distances)
glcm_df_CBIS_DDSM.to_csv("../../outputs/mamografia/matriz_features_glcm_CBIS-DDSM.csv", index=False)
glcm_df_CBIS_DDSM.head()

INBREAST

In [2]:
path_json = "C:\\Users\\mathe\\Dropbox\\PythonNotebooks\\Radiomica\\outputs\\mamografia\\extracted_metadata"
name_json = "extracted_metadata_INBREAST"
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
distances = [1, 5]

glcm_df_INBREAST = get_glcm_features(name_json, path_json, angles, properties, distances)
glcm_df_INBREAST.to_csv("../../outputs/mamografia/matriz_features_glcm_INBREAST.csv", index=False)
glcm_df_INBREAST.head()

,dissimilarity_ang_0_dist_1,dissimilarity_ang_45_dist_1,dissimilarity_ang_90_dist_1,dissimilarity_ang_135_dist_1,dissimilarity_ang_0_dist_5,dissimilarity_ang_45_dist_5,dissimilarity_ang_90_dist_5,dissimilarity_ang_135_dist_5,correlation_ang_0_dist_1,correlation_ang_45_dist_1,...,ASM_ang_135_dist_5,energy_ang_0_dist_1,energy_ang_45_dist_1,energy_ang_90_dist_1,energy_ang_135_dist_1,energy_ang_0_dist_5,energy_ang_45_dist_5,energy_ang_90_dist_5,energy_ang_135_dist_5,pathology
0,0.016037,0.017468,0.015719,0.017442,0.028800,0.029273,0.027298,0.029408,0.997370,0.997112,...,0.780883,0.884444,0.884331,0.884372,0.884333,0.883883,0.883673,0.883513,0.883676,NORMAL
1,0.017885,0.019595,0.017781,0.019532,0.031353,0.032461,0.030540,0.032441,0.997383,0.997089,...,0.743717,0.863376,0.863225,0.863291,0.863231,0.862655,0.862380,0.862244,0.862390,BENIGN
2,0.027353,0.029184,0.026601,0.029546,0.045597,0.044503,0.040502,0.045867,0.996965,0.996790,...,0.656304,0.811537,0.811385,0.811521,0.811337,0.810315,0.810302,0.810300,0.810126,NORMAL
3,0.026346,0.028427,0.025535,0.028125,0.044639,0.044735,0.039885,0.043967,0.996876,0.996582,...,0.663569,0.815822,0.815630,0.815802,0.815675,0.814642,0.814450,0.814561,0.814597,BENIGN
4,0.083185,0.091120,0.083896,0.089706,0.123119,0.134849,0.129895,0.129309,0.994243,0.993665,...,0.298135,0.555955,0.554030,0.555682,0.554339,0.547608,0.545026,0.545581,0.546017,MALIGNANT


In [ ]:
""" path_json = "../../outputs/mamografia/extracted_metadata"
name_json = "extracted_metadata_CBIS-DDSM"

list_metadata = load_json(name_json, path_json)

image_names = []
labels = []

properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']
glcm_features = [] # Cada uma das propriedades é trazida para a lista de features
angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]

for metadata in list_metadata:
    image_name = metadata['study_name']
    image_names.append(image_name)
    
    metadata_csv = metadata['metadata_csv']
    
    label = metadata_csv['pathology']
    labels.append(label)
    
    directory = Path(metadata_csv['original_image_path'])
    paths_dicom_file = list(directory.rglob("*.dcm*"))[0]
    dicom_file = dcmread(paths_dicom_file)
    
    # Carrega a matriz da imagem original
    image = dicom_file.pixel_array
    
    # Redimensiona a imagem original e salva no array
    image_8bits = (image / 256).astype(np.uint8)
    
    glcm = graycomatrix(image_8bits, distances=[1, 5], angles=angles, levels=256, symmetric=True, normed=True)
    feature = []
    glcm_props = [propery for name in properties for propery in graycoprops(glcm, name)]
    
    for glcm_props_distance in glcm_props:
        for item in glcm_props_distance:
            feature.append(item)
            
    glcm_features.append(feature)
    
columns = []
distances = ['1', '5']
angles = ['0', '45', '90', '135']

for name in properties:
    for distance in distances:
        for ang in angles:
            columns.append(name + "_ang_" + ang + "_dist_" + distance) """